In [1]:
from typing import Dict
import pandas as pd

In [2]:
historic = catalog.load('int_parts_alphavantage_etf_historic_cleansed')

2021-10-06 21:04:57,522 - kedro.io.data_catalog - INFO - Loading data from `int_parts_alphavantage_etf_historic_cleansed` (PartitionedDataSet)...


# Dividends

In [6]:
# fix ISXF
# Fix some of the top ETFs
# why are annual dividends odd numbers? 13, 5 etc

# get expense ratios for ETFs - subtract from dividend yield

In [171]:
# divide by 100 twice if pct > 10%

def extract_latest_dividend_data(historic: Dict):
    
    df_dividend = pd.DataFrame()
    df_dividend.index.name = 'symbol'

    year_ago_date = pd.to_datetime('today') - pd.to_timedelta(350, unit='d')

    for symbol, partition in historic.items():

        df = partition()
        df.date = pd.to_datetime(df.date)
        
        latest_close = df.loc[max(df.date) == df.date, 'close'].values[0]
        
        div = df[df.dividend > 0][['date', 'close', 'dividend']]


        # how often is dividend paid (most recently)
        div['last_year'] = div.date >= year_ago_date
        count_last_years_dividends = sum(div.last_year)

        # verify dividends, otherwise skip and continue remaining
        if count_last_years_dividends == 0:
            continue
        else:
            df_dividend.loc[symbol, 'annual_dividends'] = sum(div.last_year)

        # when is next dividend payment likely to be?
        df_dividend.loc[symbol, 'approx_next_dividend_date'] = min(div.date[div.last_year]) + pd.to_timedelta(365, unit='d')
       
        # no annual dividend is greater than 10%
        while (sum(div.loc[div.last_year, 'dividend']) / latest_close) > 0.10:
            div.dividend = div.dividend / 100
            
        # no annual dividend is less than 0.10%
        while (sum(div.loc[div.last_year, 'dividend']) / latest_close) < 0.001:
            div.dividend = div.dividend * 100
        
        div['pct'] = div.dividend / div.close
        
        
        
        # how much is next dividend payment likely to be? (as percentage of previous dividend close)
        df_dividend.loc[symbol, 'approx_next_dividend_pct'] = div.pct[div.date == min(div.date[div.last_year])].iloc[0]
                
        # what is estimated dividend annual yield
        df_dividend.loc[symbol, 'dividend_yield'] = sum(div.loc[div.last_year, 'dividend']) / latest_close
        
        

    return df_dividend.reset_index()

In [173]:
extract_latest_dividend_data(historic).nlargest(20, 'dividend_yield')

,symbol,annual_dividends,approx_next_dividend_date,approx_next_dividend_pct,dividend_yield
56,SEDY.LON,4.0,2021-12-10,0.005980,0.060791
45,IUKD.LON,4.0,2021-12-10,0.007952,0.057544
7,GBDV.LON,4.0,2021-11-02,0.013272,0.051561
18,IDVY.LON,4.0,2021-12-10,0.005134,0.047940
61,SHYU.LON,2.0,2021-11-12,0.023258,0.045284
11,IAPD.LON,4.0,2021-12-10,0.011097,0.044678
58,SEML.LON,2.0,2022-01-14,0.020261,0.044423
26,IHHG.LON,2.0,2021-11-12,0.023111,0.043965
63,STHS.LON,11.0,2021-11-19,0.005052,0.043799
17,IBZL.LON,4.0,2021-12-10,0.003947,0.040939


## Dividend break down

In [154]:
df_dividend = pd.DataFrame()
df_dividend.index.name = 'symbol'

year_ago_date = pd.to_datetime('today') - pd.to_timedelta(350, unit='d')

symbol = 'MIDD.LON'

partition = historic[symbol]

df = partition()
df.date = pd.to_datetime(df.date)

latest_close = df.loc[max(df.date) == df.date, 'close'].values[0]

div = df[df.dividend > 0][['date', 'close', 'dividend']]

In [155]:
# how often is dividend paid (most recently)
div['last_year'] = div.date >= year_ago_date
count_last_years_dividends = sum(div.last_year)

In [156]:
# verify dividends, otherwise skip and continue remaining
# if count_last_years_dividends == 0:
#     continue
# else:
df_dividend.loc[symbol, 'annual_dividends'] = sum(div.last_year)

# when is next dividend payment likely to be?
df_dividend.loc[symbol, 'approx_next_dividend_date'] = min(div.date[div.last_year]) + pd.to_timedelta(365, unit='d')

In [163]:
# adjust dividend amounts, readjusting for incorrect magnitudes
# div['pct'] = 100 * div.dividend / div.close  # adjusted for GBX

# no single dividend is greater than 10%
while (sum(div.loc[div.last_year, 'dividend']) / latest_close) > 0.10:
    div.dividend = div.dividend / 100  
    
# no annual dividend is less than 0.10%
while (sum(div.loc[div.last_year, 'dividend']) / latest_close) < 0.001:
    div.dividend = div.dividend * 100
    
# percentage for approximating next dividend yield
div['pct'] = 100 * div.dividend / div.close

In [164]:
div

,date,close,dividend,last_year,pct
36,2005-02-23,714.0,2.06,False,0.288515
98,2005-05-25,697.5,1.59,False,0.227957
166,2005-08-31,771.5,6.24,False,0.808814
231,2005-11-30,830.5,5.68,False,0.683925
288,2006-02-22,943.5,2.15,False,0.227875
...,...,...,...,...,...
3962,2020-09-17,1674.8,8.31,False,0.496179
4022,2020-12-10,1879.4,7.24,True,0.385229
4084,2021-03-11,2040.0,5.22,True,0.255882
4150,2021-06-17,2131.5,12.43,True,0.583157


In [165]:
dividend_yield = sum(div.loc[div.last_year, 'dividend']) / latest_close
dividend_yield

0.018865375956776228

In [166]:
# change check above to only check the yield over the past year as indicator
div

,date,close,dividend,last_year,pct
36,2005-02-23,714.0,2.06,False,0.288515
98,2005-05-25,697.5,1.59,False,0.227957
166,2005-08-31,771.5,6.24,False,0.808814
231,2005-11-30,830.5,5.68,False,0.683925
288,2006-02-22,943.5,2.15,False,0.227875
...,...,...,...,...,...
3962,2020-09-17,1674.8,8.31,False,0.496179
4022,2020-12-10,1879.4,7.24,True,0.385229
4084,2021-03-11,2040.0,5.22,True,0.255882
4150,2021-06-17,2131.5,12.43,True,0.583157


In [168]:
# how much is next dividend payment likely to be? (as percentage of previous dividend close)
df_dividend.loc[symbol, 'approx_next_dividend'] = div.pct[div.date == min(div.date[div.last_year])].iloc[0]

# what is estimated dividend annual yield
df_dividend.loc[symbol, 'dividend_yield'] = sum(div.loc[div.last_year, 'dividend']) / latest_close

df_dividend

,annual_dividends,approx_next_dividend_date,approx_next_dividend,dividend_yield
symbol,,,,
MIDD.LON,4.0,2021-12-10,0.385229,0.018865


## Extract individual stock dividends

In [137]:
ISXF = historic['MIDD.LON']()

In [138]:
ISXF.query('dividend > 0').tail(12).dividend.sum()

133.48

In [139]:
ISXF.loc[max(ISXF.date) == ISXF.date, 'close'].values[0]

2221.0

In [140]:
ISXF.query('dividend > 0')

,date,open,high,low,close,adjusted_close,volume,dividend,split_coefficient
36,2005-02-23,711.0,714.50,711.000,714.0,711.0762,430927.0,2.06,1.0
98,2005-05-25,698.0,700.00,696.500,697.5,694.6594,272323.0,1.59,1.0
166,2005-08-31,768.5,772.00,768.500,771.5,768.4201,140725.0,6.24,1.0
231,2005-11-30,835.5,836.00,828.000,830.5,827.2404,296701.0,5.68,1.0
288,2006-02-22,943.5,945.00,939.500,943.5,939.8183,392891.0,2.15,1.0
...,...,...,...,...,...,...,...,...,...
3962,2020-09-17,1666.4,1678.60,1665.000,1674.8,1674.4677,153683.0,8.31,1.0
4022,2020-12-10,1890.0,1896.60,1862.484,1879.4,1879.0990,562848.0,7.24,1.0
4084,2021-03-11,2043.0,2043.50,2033.170,2040.0,2039.7257,306513.0,5.22,1.0
4150,2021-06-17,2132.0,2133.33,2120.500,2131.5,2131.3368,589340.0,12.43,1.0


#  Scrape ETF charges (FreeTrade)

In [178]:
import requests

url = 'https://public.freetrade.io/costs_and_charges/GBR/IE00BJXRZJ40.html'

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)


table_charges = pd.read_html(r.text)

In [183]:
table_charges[0]

,0,1
0,Stamp duty,0.0000% (On execution)
1,One-off charges,0.0000%
2,Ongoing charges,0.4500%
3,Transaction costs,NaN
4,FX fee,0.0000%
5,Investment Product transaction costs,0.0900%
6,Incidental costs,0.0000%


In [3]:
import requests

url = 'https://www.justetf.com/uk/etf-profile.html?query=IE00B0M62S72&groupField=index&from=search&isin=IE00B0M62S72'
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)




In [5]:
table_charges = pd.read_html(r.text, match='Total expense ratio')

ValueError: No tables found matching pattern 'Total expense ratio'

In [ ]:
table_charges

In [207]:
r.text

'<!DOCTYPE html>\n<html prefix="og: http://ogp.me/ns#" xmlns="http://www.w3.org/1999/xhtml" xml:lang="en-GB" lang="en-GB">\n<head>\n    <script id="Cookiebot" data-culture="en" src="https://consent.cookiebot.com/uc.js"\ndata-cbid="1e28f615-f02a-4e97-9111-5f9b0a16f262" type="text/javascript" async></script>\n    <script src="/scripts/vendor/lazysizes.min.js?2.2.24" async></script>\n\n    <link rel="preconnect" href="https://www.google-analytics.com">\n    <link rel="preload" as="font" href="/fonts/OpenSans-Regular.woff2" type="font/woff2" crossorigin="anonymous">\n    <link rel="preload" as="font" href="/fonts/OpenSans-Semibold.woff2" type="font/woff2" crossorigin="anonymous">\n    <link rel="preload" as="font" href="/fonts/OpenSans-Semibold.woff2" type="font/woff2" crossorigin="anonymous">\n    <link rel="preload" as="font" href="/fonts/OpenSans-Bold.woff2" type="font/woff2" crossorigin="anonymous">\n    <link rel="preload" as="font" href="/fonts/OpenSans-ExtraBold.woff2" type="font/wo

# Test implementation of idea:
1. Extract ETFs using filter on FreeTrader stocks list
2. Filter non-Acc only
3. Scrape data from Just ETFs
    - Extract Charges
    - Extract latest price
    - Extract latest dividend
4. Calculate latest yield subtract expense ratio
5. Pick top 5 ETFs
6. 💲💲💲💲💲

## Extract ETFs & non-accumulating, based on Freetrade description

In [2]:
freetrade = catalog.load('int_csv_freetrade_cleansed')

2021-11-06 07:38:17,223 - kedro.io.data_catalog - INFO - Loading data from `int_csv_freetrade_cleansed` (CSVDataSet)...


In [3]:
freetrade.set_index('isin', inplace=True)

In [4]:
stocks_description = (freetrade.title + freetrade.long_title + freetrade.subtitle).str.upper()

In [5]:
ETFs = freetrade[
    stocks_description.str.contains(' ETF') & 
    ~stocks_description.str.contains('ACC') &
    freetrade.isa_eligible == True
]

In [6]:
ETFs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 172 entries, IE00B42Q4896 to LU0322252502
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   title                      172 non-null    object
 1   long_title                 172 non-null    object
 2   subtitle                   172 non-null    object
 3   currency                   172 non-null    object
 4   isa_eligible               172 non-null    bool  
 5   sipp_eligible              172 non-null    bool  
 6   mic                        172 non-null    object
 7   symbol                     172 non-null    object
 8   fractional_enabled         172 non-null    bool  
 9   plus_only                  172 non-null    bool  
 10  for_ireland_investors      172 non-null    bool  
 11  for_netherlands_investors  172 non-null    bool  
 12  kiid_url                   172 non-null    object
 13  stock_exchange             172 non-null    object


## Scrape data from JustETFs

In [7]:
import requests
import time
import re
from bs4 import BeautifulSoup

In [8]:
header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36 Edg/93.0.961.47",
    "X-Requested-With": "XMLHttpRequest",
    "accept-language": "en-GB,en;q=0.9,en-US;q=0.8"
    
}

In [9]:
justetfs_raw = dict()
for ISIN in ETFs.index:
    justetfs_raw[ISIN] = requests.get(
        url=f'https://www.justetf.com/en/etf-profile.html?isin={ISIN}', 
        headers=header
    )
    time.sleep(5)

In [273]:
justetfs_raw

{'IE00B42Q4896': <Response [200]>,
 'IE00B3VSSL01': <Response [200]>,
 'IE00BK5BQV03': <Response [200]>,
 'IE00BYZLWM19': <Response [200]>,
 'IE00BK74KV56': <Response [200]>,
 'IE00BD6FTQ80': <Response [200]>,
 'IE00BKTN1T54': <Response [200]>,
 'IE00BJSFQX44': <Response [200]>,
 'IE00BKW9SW28': <Response [200]>,
 'IE00B7J7TB45': <Response [200]>,
 'IE00B3F81409': <Response [200]>,
 'IE00B435CG94': <Response [200]>,
 'IE00BK5BR733': <Response [200]>,
 'IE00BNG8L385': <Response [200]>,
 'IE00BYM8JD58': <Response [200]>,
 'IE00BRHZ0398': <Response [200]>,
 'IE00B60SWT88': <Response [200]>,
 'IE00B0M63060': <Response [200]>,
 'IE00BZ1NCS44': <Response [200]>,
 'IE00B27YCF74': <Response [200]>,
 'IE00B1TXK627': <Response [200]>,
 'IE00B4L60H17': <Response [200]>,
 'IE00BCRY6557': <Response [200]>,
 'IE0032077012': <Response [200]>,
 'IE00BG143G97': <Response [200]>,
 'IE00BD8KRH84': <Response [200]>,
 'IE00B1XNHC34': <Response [200]>,
 'IE00BDFK1680': <Response [200]>,
 'IE00BDFK3H31': <Re

In [10]:
def extract_justetf_data(response):
    
    soup = BeautifulSoup(response.text, 'xml')
    
    quote_currency, latest_quote = (
        soup.find(class_ = 'infobox').find(class_ = 'val')
        .text.replace('\n', ' ').strip().split()
    )
    
    date_latest_quote = (
        soup.find(class_= 'infobox').find(class_ = 'vallabel')
        .text.replace('\n', ' ').strip().split()[-1]
    )
    
    try:
        dividend_currency, one_year_dividend = (
            soup.body.find('td', text=re.compile(r'Dividends \(last 12 months\)')).parent.find(class_ = 'val2')
            .text.split()
        )
    except:
        dividend_currency = None
        one_year_dividend = None
    
    expense_ratio, expense_ratio_frequency = (
        soup.body.find('div', text=re.compile('Total expense ratio')).parent.find(class_ = 'val')
        .text.split(None)
    )
    
    return {
        'quote_currency': quote_currency,
        'latest_quote': latest_quote,
         'date_latest_quote': date_latest_quote,
         'dividend_currency': dividend_currency,
         'one_year_dividend': one_year_dividend,
         'expense_ratio': expense_ratio,
         'expense_ratio_frequency': expense_ratio_frequency
    }
    

In [11]:
extract_justetf_data(justetfs_raw['IE00B60SWT88'])

{'quote_currency': 'GBP',
 'latest_quote': '68.79',
 'date_latest_quote': '05.11.21',
 'dividend_currency': None,
 'one_year_dividend': None,
 'expense_ratio': '0.09%',
 'expense_ratio_frequency': 'p.a.'}

In [12]:
etf_data = {isin: extract_justetf_data(response) for isin, response in justetfs_raw.items()}

In [13]:
ETF_summary = pd.DataFrame.from_dict(etf_data, orient='index', dtype='float')

ETF_summary.expense_ratio = ETF_summary.expense_ratio.str[:-1].astype(float)
ETF_summary.date_latest_quote  = pd.to_datetime(ETF_summary.date_latest_quote)

In [14]:
ETF_summary.describe(include="all")

,quote_currency,latest_quote,date_latest_quote,dividend_currency,one_year_dividend,expense_ratio,expense_ratio_frequency
count,172,172.000000,172,141,141.000000,172.000000,172
unique,1,NaN,1,1,NaN,NaN,1
top,GBP,NaN,2021-05-11 00:00:00,GBP,NaN,NaN,p.a.
freq,172,NaN,172,141,NaN,NaN,172
first,NaN,NaN,2021-05-11 00:00:00,NaN,NaN,NaN,NaN
last,NaN,NaN,2021-05-11 00:00:00,NaN,NaN,NaN,NaN
mean,NaN,42.588895,NaN,NaN,0.676596,0.303837,NaN
std,NaN,54.416860,NaN,NaN,0.780956,0.211799,NaN
min,NaN,1.360000,NaN,NaN,0.020000,0.040000,NaN
25%,NaN,8.232500,NaN,NaN,0.170000,0.100000,NaN


In [15]:
ETF_summary.info()

<class 'pandas.core.frame.DataFrame'>
Index: 172 entries, IE00B42Q4896 to LU0322252502
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   quote_currency           172 non-null    object        
 1   latest_quote             172 non-null    float64       
 2   date_latest_quote        172 non-null    datetime64[ns]
 3   dividend_currency        141 non-null    object        
 4   one_year_dividend        141 non-null    float64       
 5   expense_ratio            172 non-null    float64       
 6   expense_ratio_frequency  172 non-null    object        
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 10.8+ KB


In [16]:
ETF_summary.head()

,quote_currency,latest_quote,date_latest_quote,dividend_currency,one_year_dividend,expense_ratio,expense_ratio_frequency
IE00B42Q4896,GBP,218.98,2021-05-11,None,NaN,0.14,p.a.
IE00B3VSSL01,GBP,308.10,2021-05-11,None,NaN,0.14,p.a.
IE00BK5BQV03,GBP,68.00,2021-05-11,None,NaN,0.12,p.a.
IE00BYZLWM19,GBP,42.21,2021-05-11,GBP,1.92,0.39,p.a.
IE00BK74KV56,GBP,5.11,2021-05-11,GBP,0.03,0.17,p.a.


In [17]:
ETF_summary.date_latest_quote.unique()

array(['2021-05-11T00:00:00.000000000'], dtype='datetime64[ns]')

In [18]:
ETF_summary.query('date_latest_quote == "26.02.21"')

,quote_currency,latest_quote,date_latest_quote,dividend_currency,one_year_dividend,expense_ratio,expense_ratio_frequency


## Latest yield

In [19]:
ETF_summary.eval('dividend_yield = 100 * one_year_dividend / latest_quote', inplace=True)

In [20]:
ETF_summary.eval('net_yield = dividend_yield - expense_ratio', inplace=True)

In [21]:
ETF_summary.sort_values('net_yield', na_position='last', ascending=False).head(10)

,quote_currency,latest_quote,date_latest_quote,dividend_currency,one_year_dividend,expense_ratio,expense_ratio_frequency,dividend_yield,net_yield
IE00B652H904,GBP,15.74,2021-05-11,GBP,0.94,0.65,p.a.,5.972046,5.322046
IE00B0M63060,GBP,7.34,2021-05-11,GBP,0.41,0.40,p.a.,5.585831,5.185831
IE00B7N3YW49,GBP,73.88,2021-05-11,GBP,3.50,0.55,p.a.,4.737412,4.187412
IE00BYXVWC37,GBP,9.64,2021-05-11,GBP,0.46,0.60,p.a.,4.771784,4.171784
IE00BYZLWM19,GBP,42.21,2021-05-11,GBP,1.92,0.39,p.a.,4.548685,4.158685
IE00B5M4WH52,GBP,39.22,2021-05-11,GBP,1.76,0.50,p.a.,4.487506,3.987506
IE00B4PY7Y77,GBP,77.69,2021-05-11,GBP,3.47,0.50,p.a.,4.466469,3.966469
IE00BDFJYL11,GBP,4.92,2021-05-11,GBP,0.22,0.55,p.a.,4.471545,3.921545
IE00B14X4T88,GBP,17.55,2021-05-11,GBP,0.77,0.59,p.a.,4.387464,3.797464
IE00B810Q511,GBP,31.98,2021-05-11,GBP,1.23,0.09,p.a.,3.846154,3.756154


## Top ETFs

In [22]:
top_ETFs = ETFs[['title', 'long_title', 'subtitle', 'symbol']].join(ETF_summary[['net_yield','dividend_yield', 'expense_ratio']]).dropna()

top_ETFs.sort_values('net_yield', ascending=False).head(20)

,title,long_title,subtitle,symbol,net_yield,dividend_yield,expense_ratio
isin,,,,,,,
IE00B652H904,EM Dividends,iShares EM Dividend UCITS ETF USD (Dis.),EM Divis,SEDY,5.322046,5.972046,0.65
IE00B0M63060,UK Dividend,iShares UK Dividend UCITS ETF (Dist.),UK high dividend,IUKD,5.185831,5.585831,0.40
IE00B7N3YW49,HY Corp Bond,PIMCO Short-Term High Yield Corporate Bond Ind...,High yield corp debt,SSHY,4.187412,4.737412,0.55
IE00BYXVWC37,HY Corp Bond Hdgd,PIMCO Short-Term High Yield Corporate Bond Ind...,Hedged high yield corp debt,STHS,4.171784,4.771784,0.60
IE00BYZLWM19,AT1 Capital Bond,Invesco AT1 Capital Bond ETF GBP Hedged (Dist),AT1 Index,AT1S,4.158685,4.548685,0.39
IE00B5M4WH52,J.P.M. EM Govt Bnd,iShares J.P. Morgan EM Local Government Bond U...,Emerging markets govt debt,SEML,3.987506,4.487506,0.50
IE00B4PY7Y77,$ HY Corp Bond,iShares $ HY Corp Bond UCITS ETF USD (Dis.),High yield corp debt,SHYU,3.966469,4.466469,0.50
IE00BDFJYL11,US HY Corp Bond,iShares $ High Yield Corp Bond UCITS ETF GBP H...,Hedged high yield US corp debt,IHHG,3.921545,4.471545,0.55
IE00B14X4T88,APAC Dividend,iShares Asia Pacific Dividend UCITS ETF USD (D...,APAC dividends,IAPD,3.797464,4.387464,0.59
